In [12]:
from typing import Dict, List
import os
from synthora.agents import VanillaAgent
from synthora.messages.base import BaseMessage
from synthora.workflows import task
from synthora.agents.tot_agent import ToTAgent
from synthora.workflows.scheduler.thread_pool import ThreadPoolScheduler
from getpass import getpass
from synthora.prompts.buildin import ZeroShotCoTPrompt
from typing import Any
from synthora.messages import user
from synthora.utils.pydantic_model import get_pydantic_model
from synthora.workflows.base_task import BaseTask
from synthora.workflows.scheduler.process_pool import ProcessPoolScheduler

In [2]:
openai_api_key = getpass("Enter your API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
data = [
    "How many letters 'r' in the word 'strawberry'?",
    "9.11 and 9.9, which one is bigger?",
]

我们从最简单开始

In [6]:
@task
def generate_data(prompt: str) -> List[BaseMessage]:
    agent = VanillaAgent.default()
    _ = agent.run(prompt)
    return agent.history

In [7]:
@task
def convert(*resps: List[BaseMessage]) -> List[Dict[str, str]]:
    return [
        {
            "prompt": str(resp[0].content),
            "instruct": str(resp[1].content),
            "response": str(resp[2].content),
        }
        for resp in resps
    ]

In [9]:
flow = ThreadPoolScheduler.map(generate_data, data) >> convert
flow.run()

[{'prompt': '\nYou are an AI assistant.\n',
  'instruct': "How many letters 'r' in the word 'strawberry'?",
  'response': 'The word "strawberry" contains 3 letters \'r\'.'},
 {'prompt': '\nYou are an AI assistant.\n',
  'instruct': '9.11 and 9.9, which one is bigger?',
  'response': 'The number 9.11 is bigger than 9.9. When comparing decimal numbers, you look at the digits from left to right. Both numbers have the same whole number part (9), so you compare the decimal parts: .11 and .9. Since .11 (equivalent to .110) is greater than .9 (equivalent to .900), 9.11 is greater than 9.9.'}]

通过调整agent类型，你可以获得更准确的输出

In [11]:
@task
def generate_data_using_cot(prompt: str) -> List[BaseMessage]:
    agent = VanillaAgent.default(ZeroShotCoTPrompt)
    _ = agent.run(prompt)
    return agent.history

In [12]:
flow = ThreadPoolScheduler.map(generate_data_using_cot, data) >> convert
flow.run()

[{'prompt': '\nSolve the following problem step by step. For each step,\ncarefully explain your reasoning, include all calculations, and state any assumptions you make.\nEnsure that each step logically leads to the next, and provide a clear and concise final answer at the end.\nIf relevant, break the problem into smaller parts and address each part individually before combining the results.\n',
  'instruct': "How many letters 'r' in the word 'strawberry'?",
  'response': "To find out how many times the letter 'r' appears in the word 'strawberry', we can follow these steps:\n\n1. **Break down the word 'strawberry' into individual letters:**\n   - s, t, r, a, w, b, e, r, r, y\n\n2. **Scan through each letter and count occurrences of the letter 'r':**\n   - The sequence is: s, t, **r**, a, w, b, e, **r**, **r**, y\n\n3. **Count each appearance of 'r':**\n   - The letter 'r' first appears as the 3rd letter.\n   - The letter 'r' appears again as the 8th letter.\n   - The letter 'r' appears 

通过使用ToT Agent，你可以生成更详尽的数据

In [23]:
@task
def generate_data_using_tot(prompt: str) -> List[BaseMessage]:
    agent = ToTAgent.default(level_size=2, max_turns=15)
    resp = agent.run(prompt)
    if resp.is_err:
        return []
    return agent.history

In [24]:
hard_question = "Consider a regular octagon. How many different triangles can be formed if the octagon is placed inside a circle and we can also use the center of the circle as a vertex for the triangles? Let's think step by step."
flow = ThreadPoolScheduler.map(generate_data_using_tot, data + [hard_question])
results = flow.run()

for res in results[-1][1:]:
    print(res.content)

Consider a regular octagon. How many different triangles can be formed if the octagon is placed inside a circle and we can also use the center of the circle as a vertex for the triangles? Let's think step by step.
To solve this problem, we need to consider the structure of a regular octagon and how the additional point, the circle's center, can serve as a vertex for forming triangles. 

### Step 1: Understanding the Structure
1. **Regular Octagon**: A regular octagon has 8 vertices.
2. **Center of the Circle**: The center of the circle can be an additional vertex for forming triangles.

### Current Problem:
We need to find out how many different triangles can be formed using any of these 9 points (8 vertices of the octagon plus the center of the circle) as vertices for the triangles.

### Step 2: Determine Possible Combinations
To find the number of triangles, we will:
   - Consider all combinations of 3 points from these 9 points.
   - Calculate how many of these combinations form non

In [25]:
for res in results[1][1:]:
    print(res.content)

9.11 and 9.9, which one is bigger?
To determine which number is bigger between 9.11 and 9.9, we need to compare these two decimal numbers. 

**Think**: When comparing decimal numbers, we start by comparing the whole numbers. If the whole numbers are equal, we then compare the digits after the decimal point from left to right till we find a difference.

**Output**: Let's start by comparing the whole number parts of 9.11 and 9.9. Both numbers have the whole number part of 9, which means they are equal at this stage. 

Next, we'll compare the digits following the decimal point. We now proceed with these decimal comparisons.
**Think**: With both numbers having the whole number part equal at 9, we move to the decimal part: 

- For 9.11, the first digit after the decimal is 1.
- For 9.9, the first digit after the decimal is 9.

Understand that 9.9 can be represented as 9.90 to match the number of decimal places in 9.11 for easy comparison.

We compare:
- 11 for 9.11
- 90 for 9.90

The next s

更复杂的生成

如何打分

In [9]:
response_format = get_pydantic_model('{"score1": 0.0, "score2": 0.0}')


def score_response(
    history1: List[BaseMessage], history2: List[BaseMessage], prompt: str
) -> Dict[str, Any]:
    agent = VanillaAgent.default(
        f"You are a judger to score theses two responses. Please use score "
        f"from 0 to 10. The question is: {prompt}",
    )
    agent.model.config["response_format"] = response_format
    openai_history1 = [msg.to_openai_message() for msg in history1[1:]]
    openai_history2 = [msg.to_openai_message() for msg in history2[1:]]
    _history1 = "\n".join(
        [f"{msg['role']}: {msg['content']}" for msg in openai_history1]
    )
    _history2 = "\n".join(
        [f"{msg['role']}: {msg['content']}" for msg in openai_history2]
    )

    agent.history.append(user(f"Response 1:\n{_history1}"))
    agent.history.append(user(f"Response 2:\n{_history2}"))
    resp = agent.run("Please score the two responses.").unwrap().parsed
    result = {
        "chosen": openai_history1 if resp.score1 > resp.score2 else openai_history2,
        "rejected": openai_history2 if resp.score1 > resp.score2 else openai_history1,
        "score_chosen": resp.score1 if resp.score1 > resp.score2 else resp.score2,
        "score_rejected": resp.score2 if resp.score1 > resp.score2 else resp.score1,
    }
    return result

如何生成

In [10]:
def generate_data(system1: str, system2: str, prompt: str) -> Dict[str, Any]:
    agent1, agent2 = (
        VanillaAgent.default(system1),
        VanillaAgent.default(system2),
    )
    flow = (BaseTask(agent1.run) | BaseTask(agent2.run)).s(prompt)
    _ = flow.run()
    return score_response(agent1.history, agent2.history, prompt)

使用不同的prompt生成，但这里为了简单我是用相同的systemmessage

In [11]:
system_message = "You are an AI Assistant."
system1 = [system_message for _ in range(len(data))]
system2 = [system_message for _ in range(len(data))]

这里我们使用多进程提高生成速度

In [13]:

flow = ProcessPoolScheduler.starmap(
        BaseTask(generate_data), zip(system1, system2, data)
    )
results = flow.run()

In [14]:
for result in results:
    print(result)

{'chosen': [{'content': "How many letters 'r' in the word 'strawberry'?", 'role': 'user', 'name': 'user'}, {'content': "The word 'strawberry' contains three letters 'r'.", 'role': 'assistant', 'name': 'gpt-4o'}], 'rejected': [{'content': "How many letters 'r' in the word 'strawberry'?", 'role': 'user', 'name': 'user'}, {'content': 'The word "strawberry" contains two letters \'r\'.', 'role': 'assistant', 'name': 'gpt-4o'}], 'score_chosen': 10.0, 'score_rejected': 6.0}
{'chosen': [{'content': '9.11 and 9.9, which one is bigger?', 'role': 'user', 'name': 'user'}, {'content': '9.11 is larger than 9.9. This is because 9.9 is equivalent to 9.90, and when comparing the numbers 9.11 and 9.90, the first decimal place is the same (9), but the second decimal place shows that 11 is greater than 9.', 'role': 'assistant', 'name': 'gpt-4o'}], 'rejected': [{'content': '9.11 and 9.9, which one is bigger?', 'role': 'user', 'name': 'user'}, {'content': 'The number 9.11 is bigger than 9.9. To compare the 